In [1]:

from typing import Dict, Any, Optional, List, Annotated, TypedDict, Union
from langchain_core.messages import HumanMessage, SystemMessage
import operator

# ----- Shared State Schema -----
class AgentState(TypedDict):
    user_input: str
    supervisor_decision: Optional[str] = None
    maximo_payload: Optional[str] = None
    maximo_agent_response: Optional[str] = None
    vector_search_result: Optional[str] = None
    final_response: Optional[str] = None
    memory_chain: Annotated[List[Dict[str, Any]], operator.add] = []


In [2]:
from langchain.agents import tool
import ast
from connectors.maximo_connector import MaximoConnector

# instantiate maximo connector.
maximo_connector = MaximoConnector()

In [3]:
from pydantic import BaseModel, Field
class MaiximoPayloadInput(BaseModel):
    maximo_payload: Union[Dict, str] = Field(description="The payload to be sent to Maximo.")

In [4]:

@tool(args_schema=MaiximoPayloadInput)
def perform_maximo_operation(maximo_payload: Union[Dict, str]):
    """
    Uses the maximo tools at the llm with tools to perform the maximo operation.
    :param state: The state of the agent containing the user input and states to be updated.
    :return: A dictionary containing the Maximo payload.
    """
    # Check to see the maximo payload returned, and the response type to perform the correct action.
    if isinstance(maximo_payload, str):
        maximo_payload = ast.literal_eval(maximo_payload)

    request_type = maximo_payload.get("request_type")
    params = maximo_payload.get("params")

    if request_type.lower() == 'get':
        result = maximo_connector.get_workorder_details(params)
    elif request_type.lower() == 'post':
        result = maximo_connector.post_workorder_details(params)
    else:
        result = {
            "message": "This query is not related to Maximo."
        }

    return {
        "maximo_agent_response": result
    }


In [5]:
perform_maximo_operation.description

'Uses the maximo tools at the llm with tools to perform the maximo operation.\n:param state: The state of the agent containing the user input and states to be updated.\n:return: A dictionary containing the Maximo payload.'

In [6]:
perform_maximo_operation.name

'perform_maximo_operation'

In [7]:
perform_maximo_operation.args

{'maximo_payload': {'anyOf': [{'additionalProperties': True, 'type': 'object'},
   {'type': 'string'}],
  'description': 'The payload to be sent to Maximo.',
  'title': 'Maximo Payload'}}

In [8]:
class MaximoPayloadGeneratorInput(BaseModel):
    user_input: str = Field(description="Thing to search for")
    system_prompt: SystemMessage = Field(description="System prompt to use for generating the payload")
    llm: Any = Field(description="LLM to use for generating the payload")

@tool(args_schema=MaximoPayloadGeneratorInput)
def generate_maximo_payload(user_input, system_prompt, llm) -> Dict[str, Any]:
    """
    Generates the Maximo payload based on the user input.
    :param state: The state of the agent containing the user input and to be updated.
    :return: A dictionary containing the Maximo payload.
    """
    # Check if the user input is classified as a Maximo operation
    user_input = HumanMessage(
        content=user_input
    )
    messages = [
        system_prompt,
        user_input,
    ]

    response = llm.invoke(messages)

    # validate do a dict.
    maximo_payload = ast.literal_eval(response.content)

    return maximo_payload

In [9]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [10]:
convert_to_openai_function(perform_maximo_operation)

{'name': 'perform_maximo_operation',
 'description': 'Uses the maximo tools at the llm with tools to perform the maximo operation.\n:param state: The state of the agent containing the user input and states to be updated.\n:return: A dictionary containing the Maximo payload.',
 'parameters': {'properties': {'maximo_payload': {'anyOf': [{'additionalProperties': True,
      'type': 'object'},
     {'type': 'string'}],
    'description': 'The payload to be sent to Maximo.'}},
  'required': ['maximo_payload'],
  'type': 'object'}}

In [11]:
params1 = {
    "maximo_payload": {
        "request_type": "get",
        "params": {
            "oslc.where": 'reportdate="1998-12-31T09:00:00+00:00"',
            "oslc.select": "description,wopriority,wonum",
            "lean": "1",
            "ignorecollectionref": "1"
        }
    }
}

perform_maximo_operation.invoke(params1)

{'maximo_agent_response': [{'_rowstamp': '375928',
   'wopriority': 5,
   'description': 'Scale Calibration on Dock Mis-reading',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDAz',
   'wonum': '2003'},
  {'_rowstamp': '375661',
   'wopriority': 2,
   'description': 'Check-out Alignment of Steel Support Bracket',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDA2',
   'wonum': '2006'},
  {'_rowstamp': '375663',
   'wopriority': 5,
   'description': 'Fire Door Cable Broken',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC81MDA4',
   'wonum': '5008'},
  {'_rowstamp': '375666',
   'wopriority': 2,
   'description': 'Check-out Leaking',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDA1',
   'wonum': '2005'},
  {'_rowstamp': '375669',
   'wopriority': 3,
   'description': 'Ventilation Fan - Check-out Noise',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDAx',
   'wonum': '2001'},
  {'_rowstamp': '37567

In [12]:
params2 = {
    "maximo_payload": """{
        "request_type": "get",
        "params": {
            "oslc.where": 'reportdate="1998-12-31T09:00:00+00:00"',
            "oslc.select": "description,wopriority,wonum",
            "lean": "1",
            "ignorecollectionref": "1"
        }
    }"""
}

perform_maximo_operation.invoke(params2)

{'maximo_agent_response': [{'_rowstamp': '375703',
   'wopriority': 2,
   'description': 'Window Broken in Shipping Dept.',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC81MDAw',
   'wonum': '5000'},
  {'_rowstamp': '375721',
   'wopriority': 7,
   'description': 'Repair Damaged Conduit Feeding Generator',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8xMDA4',
   'wonum': '1008'},
  {'_rowstamp': '375726',
   'wopriority': 8,
   'description': 'Fire Door Cable Broken',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDA4',
   'wonum': '2008'},
  {'_rowstamp': '375928',
   'wopriority': 5,
   'description': 'Scale Calibration on Dock Mis-reading',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDAz',
   'wonum': '2003'},
  {'_rowstamp': '375661',
   'wopriority': 2,
   'description': 'Check-out Alignment of Steel Support Bracket',
   'href': 'http://localhost/maximo/api/os/mxapiwo/_QkVERk9SRC8yMDA2',
   'wonum': '2006'},
 

In [13]:
from langchain_ibm import ChatWatsonx

import os
from dotenv import load_dotenv
_ = load_dotenv()

payload_generator_model_params = {
            "decoding_method": "greedy",
            "max_new_tokens": 500,
            "min_new_tokens": 1,
            "temperature": 0.0,
            "top_k": 50,
            "top_p": 1,
        }

llm = ChatWatsonx(   
            model_id="meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
                url=os.environ["WATSONX_URL"],
                apikey=os.environ["IBM_CLOUD_APIKEY"],
                project_id=os.environ["WATSONX_PROJECT_ID"],
                params=payload_generator_model_params
            )

In [14]:
from langchain_ibm import ChatWatsonx

import os
from dotenv import load_dotenv
_ = load_dotenv()

#### Model using Tools
payload_generator_model_params = {
            "decoding_method": "greedy",
            "max_new_tokens": 5000,
            "min_new_tokens": 1,
            "temperature": 0.0,
            "top_k": 50,
            "top_p": 1,
        }

llm_with_tools = ChatWatsonx(   
            model_id="ibm/granite-3-8b-instruct",
                url=os.environ["WATSONX_URL"],
                apikey=os.environ["IBM_CLOUD_APIKEY"],
                project_id=os.environ["WATSONX_PROJECT_ID"],
                params=payload_generator_model_params
            )

tools = [
    generate_maximo_payload,
    perform_maximo_operation
]

llm_with_tools = llm_with_tools.bind_tools(tools)

In [15]:
user_input = "What is the status and priority of work order number 5001?"
system_prompt = SystemMessage(content="""<|begin_of_text|><|header_start|>system<|header_end|>
You are a Maximo expert. Your job is to translate human or user query into a maximo
payload that can be used to make an API Get or Post request. When you receive the human
query, you should generate a well-formed payload format. Use the examples to help you. The formats are based on whether or not 
the query is best served by a get or post request.
To help you generate the payload, use your knowledge of oslc syntax and operators to construct the well-formed payload.
The user's query may require you to select different fields. Choose the one which the user query is most closely asking for, and use it to select the correct fields to query. Only select the fields given in the list. Do not use any other fields.
<wo_fields>
[
    'acttoolcost', 'apptrequired', 'historyflag', 'aos', 'estservcost',
    'pluscismobile', 'actlabcost', 'actoutlabcost', 'estatapprlabhrs',
    'estatapprservcost', 'parentchgsstatus', 'estatapprlabcost',
    'assetlocpriority', 'ignoresrmavail', 'outtoolcost',
    'estatapproutlabhrs', '_rowstamp', 'lms', 'estatapprintlabcost',
    'istask', 'siteid', 'href', 'estatapprmatcost', 'totalworkunits',
    'suspendflow', 'status_description', 'woisswap', 'wopriority',
    'pluscloop', 'actintlabhrs', 'woacceptscharges', 'repairlocflag',
    'actmatcost', 'changedate', 'actlabhrs', 'calcpriority', 'chargestore',
    'woclass_description', 'outlabcost', 'nestedjpinprocess', 'orgid',
    'estatapprtoolcost', 'hasfollowupwork', 'phone', 'woclass',
    'actservcost', 'flowactionassist', 'ignorediavail', 'actoutlabhrs',
    'reqasstdwntime', 'estmatcost', 'supervisor', 'status',
    'inctasksinsched', 'targstartdate', 'flowcontrolled', 'ams',
    'reportdate', 'estlabhrs', 'description', 'esttoolcost', 'reportedby',
    'estatapproutlabcost', 'newchildclass', 'los', 'djpapplied',
    'estoutlabcost', 'estoutlabhrs', 'disabled', 'outmatcost',
    'actintlabcost', 'ai_usefortraining', 'estdur', 'changeby', 'worktype',
    'estintlabhrs', 'interruptible', 'estlabcost', 'estatapprintlabhrs',
    'statusdate', 'wonum', 'downtime', 'glaccount', 'workorderid',
    'milestone', 'wogroup', 'location', 'estintlabcost', 'haschildren'
]
</wo_fields>
When generating date and time related queries, use the ISO datetime format such as: "1999-02-06T00:00:00-05:00"
Once you decide on the operation type, such as Get or Post, you should generate a well-formed payload that can be provided as params to make an api call for the correct request type.
If the query does not have all the required information, use the examples below along with the information from the query to help you.
Always generate a consistent well-formed payload as a response, like in the example. The <example-get></example-get> provies making queries to the Maximo API that only retrieves data
and answers the user query. While the <example-post></example-post> provides making queries to the Maximo API that updates, modifies or changes data in the Maximo database. Make sure you
use the correct request type based on what the user is asking and format the correct payload. 
<example-get1>
user_input: What is the status, description and priority of work order number 5012?
response: {
			"request_type": "get",
			"params": {
				"oslc.where": "wonum=5012",
				"oslc.select": "wonum,description,wopriority,createdby,workorderid,status,createdate,siteid",
				"lean": "1",
				"ignorecollectionref": "1"
				}
			}
</example-get1>
<example-get2>
user_input: What was the priority and status of all work orders reported on 1998, December 22?
response: {	
	"request_type": "get",
        "params":{
		"oslc.where": 'reportdate>="1998-12-31T00:00:00+00:00" and reportdate<="1998-12-31T23:59:59+00:00"',
		"oslc.select": "wopriority,status",
		"lean": "1",
		"ignorecollectionref": "1"
		}
}
</example-get2>
<example-post>
user_input: Make a change to the priority of work order 2 and change the site to Bedford.
response: {
			"request_type": "post",
			"params": {
				"wopriority": "1",
				"siteid": "BEDFORD"
				}
			}
</example-post>
Only provide the payload that can be sent to the Maximo API. Ensure it is a valid json.
Do not provide any other information or explanation.
If the user input is not related to Maximo, send back a response with 
{
	"params": {
		"message": "This query is not related to Maximo."
	}
}
</example-post>
Now classify the type of request the user is making and generate the well-formed payload. Only output the payload and nothing else.
<|eot|><|header_start|>user<|header_end|>
<|eot|><|header_start|>assistant<|header_end|>
""")
llm = llm

# Generate the Maximo payload
params_payload_generator = {
    "user_input": user_input,
    "system_prompt": system_prompt,
    "llm": llm
}

In [16]:
type(system_prompt)

langchain_core.messages.system.SystemMessage

In [17]:
convert_to_openai_function(generate_maximo_payload)

{'name': 'generate_maximo_payload',
 'description': 'Generates the Maximo payload based on the user input.\n:param state: The state of the agent containing the user input and to be updated.\n:return: A dictionary containing the Maximo payload.',
 'parameters': {'properties': {'user_input': {'description': 'Thing to search for',
    'type': 'string'},
   'system_prompt': {'additionalProperties': True,
    'description': 'Message for priming AI behavior.\n\nThe system message is usually passed in as the first of a sequence\nof input messages.\n\nExample:\n\n    .. code-block:: python\n\n        from langchain_core.messages import HumanMessage, SystemMessage\n\n        messages = [\n            SystemMessage(\n                content="You are a helpful assistant! Your name is Bob."\n            ),\n            HumanMessage(\n                content="What is your name?"\n            )\n        ]\n\n        # Define a chat model and invoke it with the messages\n        print(model.invoke(me

In [18]:
generate_maximo_payload.invoke(params_payload_generator)

{'request_type': 'get',
 'params': {'oslc.where': 'wonum=5001',
  'oslc.select': 'status,wopriority',
  'lean': '1',
  'ignorecollectionref': '1'}}

In [29]:
from langchain.prompts import ChatPromptTemplate

user_input = "What is the status and priority of work order number 5001?"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a maximo expert. You will be given a user input and you will use the tools at your disposal to help answer that user query by retrieving the correct data using the tools at your disposal."),
    ("user", "{user_input}"),
])
chain = prompt | llm_with_tools

In [30]:
result = chain.invoke(user_input)

In [31]:
result.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'content': '',
  'additional_kwargs': {'tool_calls': [{'id': 'chatcmpl-tool-63f38b425a4b4a25ad5b295a536ed048',
     'type': 'function',
     'function': {'name': 'generate_maximo_payload',
      'arguments': '{"user_input": "What is the status and priority of work order number 5001?", "llm": "text-davinci-003", "system_prompt": {"content": "You are a Maximo expert. You will be given a user input and you will use the tools at your disposal to help answer that user query by retrieving the correct data using the tools at your disposal.\\n\\n    .. code-block:: python\\n\\n        from langchain_core.messages import HumanMessage\\n\\n        messages = [\\n            HumanMessage(\\n                content=\\"What is the status and priority of work order number 5001?\\"\\n            )\\n        ]\\n\\n        # Define a chat model and invoke it with the messages\\n        print(model.in